<a href="https://colab.research.google.com/github/Ivan8Garcia/Proyecto_NoCountry/blob/main/Vuelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

datos=pd.read_csv("/content/flight_delays.csv")
datos.shape

(1747627, 16)

In [33]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747627 entries, 0 to 1747626
Data columns (total 16 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   FlightID            int64 
 1   Airline             object
 2   FlightNumber        int64 
 3   Origin              object
 4   Destination         object
 5   ScheduledDeparture  object
 6   ActualDeparture     object
 7   ScheduledArrival    object
 8   ActualArrival       object
 9   DelayMinutes        int64 
 10  DelayReason         object
 11  Cancelled           bool  
 12  Diverted            bool  
 13  AircraftType        object
 14  TailNumber          object
 15  Distance            int64 
dtypes: bool(2), int64(4), object(10)
memory usage: 190.0+ MB


In [34]:
datos.isnull().sum()

,0
FlightID,0
Airline,0
FlightNumber,0
Origin,0
Destination,0
ScheduledDeparture,0
ActualDeparture,0
ScheduledArrival,0
ActualArrival,0
DelayMinutes,0


In [35]:
datos.fillna(0,inplace=True)

In [36]:
datos.isnull().sum()

,0
FlightID,0
Airline,0
FlightNumber,0
Origin,0
Destination,0
ScheduledDeparture,0
ActualDeparture,0
ScheduledArrival,0
ActualArrival,0
DelayMinutes,0


In [37]:
datos.head()

,FlightID,Airline,FlightNumber,Origin,Destination,ScheduledDeparture,ActualDeparture,ScheduledArrival,ActualArrival,DelayMinutes,DelayReason,Cancelled,Diverted,AircraftType,TailNumber,Distance
0,1,United,4558,ORD,MIA,2024-09-01 08:11,2024-09-01 08:30,2024-09-01 12:11,2024-09-01 12:19,8,Weather,True,False,Boeing 737,N71066,1031
1,2,Delta,8021,LAX,MIA,2024-09-01 10:25,2024-09-01 10:41,2024-09-01 13:25,2024-09-01 13:27,2,Air Traffic Control,True,True,Airbus A320,N22657,1006
2,3,Southwest,7520,DFW,SFO,2024-09-01 16:53,2024-09-01 17:05,2024-09-01 17:53,2024-09-01 18:07,14,Weather,True,True,Boeing 737,N95611,2980
3,4,Delta,2046,ORD,BOS,2024-09-01 14:44,2024-09-01 15:04,2024-09-01 18:44,2024-09-01 18:34,-10,0,False,False,Boeing 777,N90029,1408
4,5,Delta,6049,LAX,SEA,2024-09-01 01:51,2024-09-01 02:08,2024-09-01 05:51,2024-09-01 06:15,24,Air Traffic Control,False,True,Boeing 737,N27417,2298


##ELIMINACION DE COLUMNAS PARA MODELO MVP, SOLO NECESITAREMOS "Aerolínea","Origen", "Destino", "Fecha/Hora", "Retraso".

In [38]:
datos = datos[['Airline', 'Origin', 'Destination', 'ScheduledDeparture',]]
datos = datos.rename(columns={'Airline': 'Aerolínea', 'Origin': 'Origen', 'Destination': 'Destino', 'ScheduledDeparture': 'Fecha/Hora'})
datos['Fecha/Hora'] = pd.to_datetime(datos['Fecha/Hora'])

# Add the day of the week column in Spanish
# First, get the day name in English
datos['Dia_semana_temp'] = datos['Fecha/Hora'].dt.day_name()

# Map to Spanish
spanish_days = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

datos['Dia_semana'] = datos['Dia_semana_temp'].map(spanish_days)

# Drop the temporary column
datos.drop('Dia_semana_temp', axis=1, inplace=True)

datos['Fecha'] = datos['Fecha/Hora'].dt.date
datos['Hora'] = datos['Fecha/Hora'].dt.time
datos.drop('Fecha/Hora', axis=1, inplace=True)

datos.head()

,Aerolínea,Origen,Destino,Dia_semana,Fecha,Hora
0,United,ORD,MIA,Domingo,2024-09-01,08:11:00
1,Delta,LAX,MIA,Domingo,2024-09-01,10:25:00
2,Southwest,DFW,SFO,Domingo,2024-09-01,16:53:00
3,Delta,ORD,BOS,Domingo,2024-09-01,14:44:00
4,Delta,LAX,SEA,Domingo,2024-09-01,01:51:00


In [39]:
import pandas as pd

# Re-reading the data to ensure DelayMinutes is present
temp_datos = pd.read_csv("/content/flight_delays.csv")

# Fill NaN values in DelayReason as done previously, or any other NaNs that might affect numerical columns
temp_datos.fillna(0, inplace=True)

# Select the required columns including DelayMinutes
datos = temp_datos[['Airline', 'Origin', 'Destination', 'ScheduledDeparture', 'DelayMinutes']].copy()

# Rename columns to Spanish
datos = datos.rename(columns={
    'Airline': 'Aerolínea',
    'Origin': 'Origen',
    'Destination': 'Destino',
    'ScheduledDeparture': 'Fecha/Hora',
    'DelayMinutes': 'Retraso_minutos'
})

# Convert 'Fecha/Hora' to datetime
datos['Fecha/Hora'] = pd.to_datetime(datos['Fecha/Hora'])

# Add the day of the week column in Spanish
datos['Dia_semana_temp'] = datos['Fecha/Hora'].dt.day_name()
spanish_days = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}
datos['Dia_semana'] = datos['Dia_semana_temp'].map(spanish_days)

# Create the binary 'retrasado' column based on 'Retraso_minutos'
datos['Retrasado'] = (datos['Retraso_minutos'] > 15).astype(int)

# Split Fecha/Hora into Fecha and Hora
datos['Fecha'] = datos['Fecha/Hora'].dt.date
datos['Hora'] = datos['Fecha/Hora'].dt.time

# Drop temporary and original Fecha/Hora column
datos.drop(columns=['Fecha/Hora', 'Dia_semana_temp'], inplace=True)

# Ensure the requested column order, placing 'Retraso_minutos' and 'Retrasado' after 'Hora'
datos = datos[['Aerolínea', 'Origen', 'Destino', 'Dia_semana', 'Fecha', 'Hora', 'Retraso_minutos', 'Retrasado']]

datos.head()

,Aerolínea,Origen,Destino,Dia_semana,Fecha,Hora,Retraso_minutos,Retrasado
0,United,ORD,MIA,Domingo,2024-09-01,08:11:00,8,0
1,Delta,LAX,MIA,Domingo,2024-09-01,10:25:00,2,0
2,Southwest,DFW,SFO,Domingo,2024-09-01,16:53:00,14,0
3,Delta,ORD,BOS,Domingo,2024-09-01,14:44:00,-10,0
4,Delta,LAX,SEA,Domingo,2024-09-01,01:51:00,24,1


Entregables deseados

Notebook (Jupyter/Colab) del equipo de Data Science, que contenga:

Exploración y limpieza de datos (EDA);

Creación de variables relevantes (ej.: hora del vuelo, día de la semana, aeropuerto de origen/destino, aerolínea);

Entrenamiento de un modelo predictivo (ej.: Logistic Regression, Random Forest);

Evaluación del desempeño (Accuracy, Precision, Recall, F1-score);

Exportación del modelo serializado (joblib/pickle).

In [40]:
datos.head()

,Aerolínea,Origen,Destino,Dia_semana,Fecha,Hora,Retraso_minutos,Retrasado
0,United,ORD,MIA,Domingo,2024-09-01,08:11:00,8,0
1,Delta,LAX,MIA,Domingo,2024-09-01,10:25:00,2,0
2,Southwest,DFW,SFO,Domingo,2024-09-01,16:53:00,14,0
3,Delta,ORD,BOS,Domingo,2024-09-01,14:44:00,-10,0
4,Delta,LAX,SEA,Domingo,2024-09-01,01:51:00,24,1


In [41]:
datos['Retraso_minutos'] = datos['Retraso_minutos'].astype('float64')
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747627 entries, 0 to 1747626
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Aerolínea        object 
 1   Origen           object 
 2   Destino          object 
 3   Dia_semana       object 
 4   Fecha            object 
 5   Hora             object 
 6   Retraso_minutos  float64
 7   Retrasado        int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 106.7+ MB


In [42]:
# Convert 'Hora' from datetime.time object to a float representing total hours
datos['Hora'] = datos['Hora'].apply(lambda x: x.hour + x.minute/60 + x.second/3600)

datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747627 entries, 0 to 1747626
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Aerolínea        object 
 1   Origen           object 
 2   Destino          object 
 3   Dia_semana       object 
 4   Fecha            object 
 5   Hora             float64
 6   Retraso_minutos  float64
 7   Retrasado        int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 106.7+ MB


In [43]:
datos.head()

,Aerolínea,Origen,Destino,Dia_semana,Fecha,Hora,Retraso_minutos,Retrasado
0,United,ORD,MIA,Domingo,2024-09-01,8.183333,8.0,0
1,Delta,LAX,MIA,Domingo,2024-09-01,10.416667,2.0,0
2,Southwest,DFW,SFO,Domingo,2024-09-01,16.883333,14.0,0
3,Delta,ORD,BOS,Domingo,2024-09-01,14.733333,-10.0,0
4,Delta,LAX,SEA,Domingo,2024-09-01,1.850000,24.0,1


In [44]:
datos.tail()

,Aerolínea,Origen,Destino,Dia_semana,Fecha,Hora,Retraso_minutos,Retrasado
1747622,United,DFW,MIA,Domingo,2024-09-01,12.766667,-5.0,0
1747623,United,ATL,SEA,Domingo,2024-09-01,20.716667,20.0,1
1747624,Delta,JFK,SFO,Domingo,2024-09-01,3.433333,-6.0,0
1747625,Delta,JFK,SEA,Domingo,2024-09-01,8.066667,16.0,1
1747626,United,LAX,MIA,Domingo,2024-09-01,12.766667,19.0,1


##**Metodo get_dummies**

In [45]:
#Todas estas columnas son categoricas (OBJ), para que se puedan hacer buenos modelos hay que codificarlas
categoricas= ["Aerolínea","Origen","Destino","Dia_semana"]

pd.get_dummies(data=datos,columns=categoricas,dtype=int).head(3)

,Fecha,Hora,Retraso_minutos,Retrasado,Aerolínea_American Airlines,Aerolínea_Delta,Aerolínea_Southwest,Aerolínea_United,Origen_ATL,Origen_DFW,Origen_JFK,Origen_LAX,Origen_ORD,Destino_BOS,Destino_JFK,Destino_MIA,Destino_SEA,Destino_SFO,Dia_semana_Domingo
0,2024-09-01,8.183333,8.0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1
1,2024-09-01,10.416667,2.0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1
2,2024-09-01,16.883333,14.0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1


In [46]:
datos_codificados=pd.get_dummies(data=datos,columns=categoricas,dtype=int)
datos_codificados.sample(3)

,Fecha,Hora,Retraso_minutos,Retrasado,Aerolínea_American Airlines,Aerolínea_Delta,Aerolínea_Southwest,Aerolínea_United,Origen_ATL,Origen_DFW,Origen_JFK,Origen_LAX,Origen_ORD,Destino_BOS,Destino_JFK,Destino_MIA,Destino_SEA,Destino_SFO,Dia_semana_Domingo
1290298,2024-09-01,1.766667,17.0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1
1365770,2024-09-01,20.600000,-7.0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1
404879,2024-09-01,19.466667,18.0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1


In [ ]:
#usaremos el metodo de las correlaciones porque en las graficas cuando usamos kde,las kde se parecen mucho
#para ello usaremos el metodo de correlaciones para comprobar matematicamente cuanto se parecen
#aqui veremos la correlacion entre el horario de llega y de salida
datos[["ScheduledArrival","ActualArrival"]].corr()

In [47]:
datos_codificados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747627 entries, 0 to 1747626
Data columns (total 19 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Fecha                        object 
 1   Hora                         float64
 2   Retraso_minutos              float64
 3   Retrasado                    int64  
 4   Aerolínea_American Airlines  int64  
 5   Aerolínea_Delta              int64  
 6   Aerolínea_Southwest          int64  
 7   Aerolínea_United             int64  
 8   Origen_ATL                   int64  
 9   Origen_DFW                   int64  
 10  Origen_JFK                   int64  
 11  Origen_LAX                   int64  
 12  Origen_ORD                   int64  
 13  Destino_BOS                  int64  
 14  Destino_JFK                  int64  
 15  Destino_MIA                  int64  
 16  Destino_SEA                  int64  
 17  Destino_SFO                  int64  
 18  Dia_semana_Domingo           int64  
dtype

In [48]:
df=datos_codificados.drop(columns=["Fecha"])
df

,Hora,Retraso_minutos,Retrasado,Aerolínea_American Airlines,Aerolínea_Delta,Aerolínea_Southwest,Aerolínea_United,Origen_ATL,Origen_DFW,Origen_JFK,Origen_LAX,Origen_ORD,Destino_BOS,Destino_JFK,Destino_MIA,Destino_SEA,Destino_SFO,Dia_semana_Domingo
0,8.183333,8.0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1
1,10.416667,2.0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1
2,16.883333,14.0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1
3,14.733333,-10.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1
4,1.850000,24.0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1747622,12.766667,-5.0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1
1747623,20.716667,20.0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1
1747624,3.433333,-6.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1
1747625,8.066667,16.0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1


In [49]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split

In [50]:
#primero separamos nuestras columnas para tener las columnas de prueba y entrenamiento
X= df.drop(["Retraso_minutos"],axis=1)
y=df["Retraso_minutos"]

#segundo,empezaremos a segmentar nuestras X_train,etc...
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state=42)

baseline=DummyRegressor()
baseline.fit(X_train,y_train)

DummyRegressor()

In [51]:
#ahora vamos a evaluar el modelo usando metricas
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

y_pred_dummy=baseline.predict(X_test)

def calcular_regresion (y_test,y_pred):
  rmse=mean_squared_error(y_test,y_pred)
  mae=mean_absolute_error(y_test,y_pred)
  r2= r2_score(y_test,y_pred)
  metricas={
      "RMSE":round(rmse**(1/2),4),
      "MAE":round(mae,4),
      "R2":round(r2,4)
  }
  return metricas

In [52]:
resultados_baseline=calcular_regresion(y_test,y_pred_dummy)
resultados_baseline

{'RMSE': 11.8311, 'MAE': 10.2462, 'R2': -0.0}

## Interpretación de Métricas de Regresión

*   **Interpretación:** Mide la magnitud promedio de los errores del modelo. La unidad del RMSE es la misma que la de la variable objetivo (en tu caso, los minutos de retraso). Valores más bajos indican un mejor ajuste del modelo a los datos.
*   **¿Qué es un 'buen' valor?** No hay un umbral fijo para un 'buen' RMSE, ya que depende en gran medida del contexto y la escala de tus datos. Sin embargo, un RMSE significativamente menor que la desviación estándar de la variable objetivo suele ser una buena señal. Es una métrica sensible a errores grandes (outliers) debido al cuadrado de los errores.
*   **¿Es una buena métrica?** Sí, es ampliamente utilizada. penaliza más los errores grandes, lo que puede ser deseable si los errores grandes son particularmente problemáticos en tu aplicación. Su principal desventaja es que, al ser sensible a outliers, un solo error muy grande puede inflar el RMSE.

### 2. Error Absoluto Medio (MAE - Mean Absolute Error)

*   **¿Qué es?** El MAE es el promedio de la magnitud de los errores. A diferencia del RMSE, no eleva al cuadrado los errores, sino que toma su valor absoluto.
*   **Interpretación:** También mide la magnitud promedio de los errores del modelo. Su unidad es la misma que la de la variable objetivo. Valores más bajos indican un mejor rendimiento.
*   **¿Qué es un 'buen' valor?** Similar al RMSE, depende de la escala de tus datos. Un MAE menor que el rango de tu variable objetivo y, si es posible, menor que un umbral de error aceptable en tu dominio, es deseable.
*   **¿Es una buena métrica?** Sí, es otra métrica muy común. Es más robusta a los outliers que el RMSE porque no penaliza los errores grandes de forma cuadrática. Ofrece una interpretación más directa de la magnitud promedio del error en las unidades originales de la variable objetivo. Si prefieres que todos los errores contribuyan por igual a la métrica, MAE es una buena elección.

### 3. Coeficiente de Determinación (R2 - R-squared)

*   **¿Qué es?** El R2 es una medida de la proporción de la varianza en la variable dependiente que es predecible a partir de las variables independientes. En términos más simples, indica qué tan bien se ajustan las predicciones de tu modelo a los valores reales.
*   **Interpretación:** El valor de R2 varía entre 0 y 1 (o puede ser negativo para modelos muy malos, como en tu caso para el baseline).:
    *   **R2 = 0:** El modelo no explica ninguna de la variabilidad de la variable objetivo. Las predicciones del modelo no son mejores que simplemente usar el promedio de los valores reales.
    *   **R2 = 1:** El modelo explica el 100% de la variabilidad de la variable objetivo. El modelo predice perfectamente los valores reales.
    *   **R2 negativo:** Esto sucede cuando el modelo que has ajustado es peor que un modelo que simplemente predice la media de la variable dependiente para todas las observaciones. En tu caso, `R2: -0.0` para el `DummyRegressor` es esperado, ya que este modelo trivial solo predice la media, y la forma en que se calcula R2 lo hace ligeramente negativo si la suma de cuadrados de los residuos es mayor que la suma de cuadrados totales.
*   **¿Qué es un 'buen' valor?** En general, cuanto más cercano a 1, mejor. Sin embargo, un R2 alto no siempre significa que el modelo sea práctico, especialmente si se ha sobreajustado a los datos de entrenamiento. Un 'buen' R2 también es contextual y depende de la complejidad del problema. En ciencias sociales, un R2 de 0.3 puede ser considerado bueno, mientras que en física, se esperan valores mucho más altos.
*   **¿Es una buena métrica?** Sí, es muy útil para entender la bondad de ajuste global del modelo. La precaución principal es no usarlo como la única métrica, ya que un R2 alto no garantiza un modelo preciso o sin sesgos, y puede ser engañoso en presencia de muchos predictores.

### Resumen para tus resultados `{'RMSE': 11.8311, 'MAE': 10.2462, 'R2': -0.0}`:

Estos resultados son de tu modelo *baseline* (`DummyRegressor`). Un modelo *baseline* es un punto de partida simple y sirve para comparar. Al ser un `DummyRegressor`, es normal que obtengas un R2 muy bajo o negativo, ya que es un modelo muy básico que probablemente predice solo la media de los minutos de retraso. El MAE y RMSE de tu baseline te indican el error promedio de este modelo trivial.

Para saber si un futuro modelo es bueno, querrías ver:

*   **RMSE y MAE:** Significativamente más bajos que los obtenidos con el modelo *baseline*.
*   **R2:** Un valor positivo y, idealmente, cuanto más cercano a 1, mejor (aunque con cautela y comparando con otros modelos).

In [53]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calcular_metricas_regresion(y_test, y_pred):

    rmse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    metricas = {
        'Raíz del Error Cuadrático Medio': round(rmse, 4),
        'Error Absoluto Medio': round(mae, 4),
        'R2 Score': round(r2, 4)
    }

    return metricas

In [54]:
from sklearn.dummy import DummyRegressor

estrategias = ["mean", "median", ("quantile", 0.25), ("constant", 10.0)]

resultados = {}

for estrategia in estrategias:
    if isinstance(estrategia, tuple):
        estrategia, valor = estrategia
        if estrategia == "quantile":
            model_dummy = DummyRegressor(strategy=estrategia, quantile=valor)
        else:
            model_dummy = DummyRegressor(strategy=estrategia, constant=valor)
    else:
        model_dummy = DummyRegressor(strategy=estrategia)

    model_dummy.fit(X_train, y_train)

    y_pred_dummy = model_dummy.predict(X_test)

    metricas = calcular_metricas_regresion(y_test, y_pred_dummy)

    resultados[estrategia] = metricas

for estrategia, metricas in resultados.items():
    print(f"Estrategia: {estrategia}")
    for metrica, valor in metricas.items():
        print(f"{metrica}: {valor}")
    print("\n")


Estrategia: mean
Raíz del Error Cuadrático Medio: 139.9744
Error Absoluto Medio: 10.2462
R2 Score: -0.0


Estrategia: median
Raíz del Error Cuadrático Medio: 139.9737
Error Absoluto Medio: 10.2459
R2 Score: -0.0


Estrategia: quantile
Raíz del Error Cuadrático Medio: 239.4181
Error Absoluto Medio: 12.6613
R2 Score: -0.7105


Estrategia: constant
Raíz del Error Cuadrático Medio: 139.9737
Error Absoluto Medio: 10.2459
R2 Score: -0.0




##Metodo Random Forest

In [55]:
#los resultados son en minutos; aqui vemos que el modelo generalizó mejor
#RMSE= 6.51min, MAE=5.48min, 'R2': 0.6966%
from sklearn.ensemble import RandomForestRegressor

modelo= RandomForestRegressor(max_depth=5,random_state=42)
modelo.fit(X_train,y_train)
y_pred=modelo.predict(X_test)
resultados_rf= calcular_regresion(y_test,y_pred)
resultados_rf

{'RMSE': 6.5167, 'MAE': 5.4841, 'R2': 0.6966}

## **Cross Validation**

In [56]:
from sklearn.model_selection import KFold,cross_validate

scoring={
    "RMSE":"neg_root_mean_squared_error",
    "MAE": "neg_mean_absolute_error",
    "R2": "r2"
}
cv= KFold(n_splits=5,shuffle=True,random_state=42)
cv_resultados= cross_validate(modelo,X_train,y_train,scoring=scoring,cv=cv)
cv_resultados


{'fit_time': array([130.18211007, 124.30460167, 124.36220765, 125.53684926,
        121.8940239 ]),
 'score_time': array([0.69425941, 0.64492178, 0.66682744, 0.61802268, 0.66176414]),
 'test_RMSE': array([-6.5154207 , -6.51836522, -6.51115025, -6.52991252, -6.51376231]),
 'test_MAE': array([-5.48702873, -5.48667041, -5.47918971, -5.49948697, -5.48095653]),
 'test_R2': array([0.69695696, 0.69647215, 0.69687231, 0.69561526, 0.69664397])}

In [57]:
len(df.columns)

18

In [58]:
modelo.feature_importances_

array([5.34647601e-05, 9.99914155e-01, 1.99111706e-06, 2.10112890e-06,
       2.33846664e-06, 2.73430593e-06, 2.93984214e-06, 2.01272307e-06,
       2.39347926e-06, 2.55769617e-06, 1.82750056e-06, 2.88086839e-06,
       1.95039727e-06, 1.72700450e-06, 2.35752009e-06, 2.56773879e-06,
       0.00000000e+00])

In [59]:
importances= modelo.feature_importances_

In [62]:
feature_importances=pd.DataFrame({"Features":X.columns,"Importances":(importances*100).round(2)}).sort_values("Importances",ascending=False)
feature_importances

,Features,Importances
1,Retrasado,99.99
0,Hora,0.01
2,Aerolínea_American Airlines,0.00
3,Aerolínea_Delta,0.00
4,Aerolínea_Southwest,0.00
5,Aerolínea_United,0.00
6,Origen_ATL,0.00
7,Origen_DFW,0.00
8,Origen_JFK,0.00
9,Origen_LAX,0.00


In [69]:
resultados=pd.DataFrame(index=["RMSE","MAE","R2"])
model_features=RandomForestRegressor(max_depth=5,random_state=42)
ct_features= [i if i !=0 else 1 for i in range(0,17,4)]

for i in ct_features:
  selected_features= feature_importances["Features"].values[:i]
  X_train_sel= X_train[selected_features]
  X_test_sel= X_test[selected_features]
  model_features.fit(X_train_sel,y_train)
  y_pred=model_features.predict(X_test_sel)
  metricas=calcular_regresion(y_test,y_pred)
  resultados[i]= list(metricas.values())

resultados

,1,4,8,12,16
RMSE,6.5166,6.5166,6.5166,6.5166,6.5167
MAE,5.4838,5.4840,5.4840,5.4840,5.4841
R2,0.6966,0.6966,0.6966,0.6966,0.6966


In [70]:
resultados=pd.DataFrame(index=["RMSE","MAE","R2"])
model_features=RandomForestRegressor(max_depth=5,random_state=42)
ct_features=  range(1,5)

for i in ct_features:
  selected_features= feature_importances["Features"].values[:i]
  X_train_sel= X_train[selected_features]
  X_test_sel= X_test[selected_features]
  model_features.fit(X_train_sel,y_train)
  y_pred=model_features.predict(X_test_sel)
  metricas=calcular_regresion(y_test,y_pred)
  resultados[i]= list(metricas.values())

resultados

,1,2,3,4
RMSE,6.5166,6.5166,6.5166,6.5166
MAE,5.4838,5.4840,5.4840,5.4840
R2,0.6966,0.6966,0.6966,0.6966


In [27]:
#from sklearn.ensemble import RandomForestClassifier
#rf= RandomForestClassifier(max_depth = 10, random_state=42)
#rf.fit(pd.DataFrame(X_train), y_train)
#rf.score(X_test,y_test)

##**Metodo OneHotEnconder**

In [68]:
#from sklearn.compose import make_column_transformer
#from sklearn.preprocessing import OneHotEncoder

#categoricas= ["Aerolínea","Origen","Destino","Dia_semana"]

#one_hot_enc = make_column_transformer(
    #(OneHotEncoder(handle_unknown='ignore'),
    #categoricas),
    #remainder='passthrough',
    #sparse_threshold=0,
    #force_int_remainder_cols=False)

#datos = one_hot_enc.fit_transform(datos)
#datos = pd.DataFrame(datos, columns=one_hot_enc.get_feature_names_out())
#datos